In [29]:
import traitlets
import ipywidgets as w
import ipyvuetify as v
from spectate import mvc
from traitlets import TraitType
from traitlets import HasTraits, observe

In [37]:
class Mutable(TraitType):
    """A base class for mutable traits using Spectate"""

    _model_type = None
    _event_type = "change"

    def instance_init(self, obj):
        default = self._model_type()

        @mvc.view(default)
        def callback(default, events):
            change = dict(
                new=getattr(obj, self.name),
                name=self.name,
                type=self._event_type,
            )
            obj.notify_change(change)

        setattr(obj, self.name, default)

In [38]:
class MutableDict(Mutable):
    """A mutable dictionary trait"""

    _model_type = mvc.Dict

In [25]:
class MutableList(Mutable):
    """A mutable dictionary trait"""

    _model_type = mvc.List

    def _make_change(self, model, events):
        old, new = {}, {}
        for e in events:
            old[e["key"]] = e["old"]
            new[e["key"]] = e["new"]
        return {"value": model, "old": old, "new": new}

In [39]:
class MyObject(HasTraits):
    mutable_dict = MutableDict()

    @observe("mutable_dict", type="mutation")
    def track_mutations_from_method(self, change):
        print("method observer:", change)

In [61]:
overview_template = """
<template>
    <v-expansion-panels accordion>
        <jupyter-widget v-for="acc in accounts" :widget="acc" />
    </v-expansion-panels>
</template>
"""

In [69]:
account_expansion_panel_template = """
<v-expansion-panel>
     <v-expansion-panel-header>{{name}}</v-expansion-panel-header>
     <v-expansion-panel-content>
         <v-container>
             <v-row>
                 <v-col>
                     <v-row>Einzahlungen</v-row>
                     <v-row v-for="asset in assets">
                         <jupyter-widget :widget="asset" />
                     </v-row>
                </v-col>
                 <v-col >
                     <v-row>Auszahlungen</v-row>
                     <v-row v-for="liability in liabilities">
                         <jupyter-widget :widget="liabilitiy" />
                     </v-row>
                 </v-col>
             </v-row>
        </v-container>
    </v-expansion-panel-content>
</v-expansion-panel>
"""

In [70]:
order_card_template = """
<template>
    <v-card>
        <v-card-title>{{name}}</v-card-title>
    <v-card-actions>
      <v-btn
        text
      >
        Edit
      </v-btn>

      <v-spacer></v-spacer>

      <v-btn
        @click="show_details = !show_details"
      >
          Details
        <v-icon>{{ show_details ? 'mdi-chevron-up' : 'mdi-chevron-down' }}</v-icon>
      </v-btn>
    </v-card-actions>

    <v-expand-transition>
      <div v-show="show_details">
        <v-divider></v-divider>

        <v-card-text>
          Details ...
        </v-card-text>
      </div>
    </v-expand-transition>
    </v-card>
</template>
"""

In [71]:
class OrderCard(v.VuetifyTemplate):
    name = traitlets.Unicode().tag(sync=True)
    id_ = traitlets.Unicode().tag(sync=True)
    show_details = traitlets.Bool(False).tag(sync=True)
    
    
    @traitlets.default("template")
    def _template(self):
        return order_card_template

In [68]:
class AccountExpansionPanel(v.VuetifyTemplate):
    name = traitlets.Unicode().tag(sync=True)
    
    assets = MutableDict().tag(sync=True, **w.widget_serialization)
    liabilities = MutableDict().tag(sync=True, **w.widget_serialization)
    
    @traitlets.default("template")
    def _template(self):
        return account_expansion_panel_template

In [65]:
class Overview(v.VuetifyTemplate):
    accounts = MutableDict().tag(sync=True, **w.widget_serialization)
    
    
    @traitlets.default("template")
    def _template(self):
        return overview_template
    
    @observe("accounts", type="mutation")
    def track_mutations_from_method(self, change):
        print("method observer:", change)

In [78]:
overview = Overview()
overview

Overview(accounts={}, template='\n<template>\n    <v-expansion-panels accordion>\n        <jupyter-widget v-fo…

In [82]:
overview.accounts[0] = AccountExpansionPanel(name="Konto #2")

In [80]:
overview.accounts[1].assets[0] = OrderCard(name="Auftrag #1")

In [75]:
overview.accounts[1].assets[0].name = "Dauerauftrag"

In [81]:
overview.accounts[1].assets.pop(0)

OrderCard(name='Auftrag #1', template='\n<template>\n    <v-card>\n        <v-card-title>{{name}}</v-card-titl…